In [ ]:
import polars as pl
import pandas as pd

In [ ]:
# --- Rutas (Asegúrate que existan) ---
RUTA_DATOS_INPUT = './datos/'
RUTA_DATOS_OUTPUT = './datos/output/'
# Crear directorio de output si no existe
import os
if not os.path.exists(RUTA_DATOS_OUTPUT):
    os.makedirs(RUTA_DATOS_OUTPUT)
    print(f"Directorio creado: {RUTA_DATOS_OUTPUT}")

In [ ]:
# --- 1. Cargar los archivos ---
try:
    # Usar Polars para DIM_TIENDA y Meta_venta también para consistencia
    dim_tienda_lf = pl.scan_csv(RUTA_DATOS_INPUT + 'DIM_TIENDA.csv')
    lf_venta = pl.scan_csv(RUTA_DATOS_INPUT + 'venta.csv') # Asume encabezados TIENDA_ID, MES_ID, VENTA_TOTAL
    meta_venta_lf = pl.scan_csv(RUTA_DATOS_INPUT + 'Meta_venta.csv')
    print("Archivos CSV escaneados exitosamente con Polars.")
except Exception as e:
    print(f"Error al escanear archivos: {e}")
    raise

# Verificar columnas en lf_venta para asegurar que VENTA_TOTAL y MES_ID existen
schema_venta = lf_venta.collect_schema()
if "VENTA_TOTAL" not in schema_venta.names() or "MES_ID" not in schema_venta.names() or "TIENDA_ID" not in schema_venta.names():
    print(f"Error: Columnas esperadas no encontradas en venta.csv. Columnas detectadas: {schema_venta.names()}")
    raise ValueError("Columnas faltantes en venta.csv")


# Filtrar solo tiendas de entrenamiento
dim_tienda_train_lf = dim_tienda_lf.filter(pl.col("DATASET") == "TRAIN")


In [ ]:
# --- 2. Procesar lf_venta para obtener ventas agregadas ---
# Definir el periodo fijo: Enero 2023 a Julio 2024
# Asumimos que MES_ID es un entero AAAAMM (ej. 202308)
MES_INICIO_PERIODO = 202301 # Enero 2023
MES_FIN_PERIODO = 202407   # Julio 2024

print(f"Periodo de ventas seleccionado: de {MES_INICIO_PERIODO} a {MES_FIN_PERIODO}")

# Filtrar por el periodo y calcular promedio
# Primero, asegurar que MES_ID sea del tipo correcto para la comparación (ej. Int64)
# Si MES_ID ya es numérico en el CSV, Polars usualmente lo infiere bien.
# Si es string, hay que convertirlo: .with_columns(pl.col("MES_ID").cast(pl.Int64))

lf_venta_procesada = (
    lf_venta
    .filter(
        (pl.col("MES_ID") >= MES_INICIO_PERIODO) & (pl.col("MES_ID") <= MES_FIN_PERIODO)
    )
    .group_by("TIENDA_ID")
    .agg(
        pl.col("VENTA_TOTAL").mean().alias("VENTA_PROMEDIO_MENSUAL"),
        pl.count().alias("N_MESES_CON_VENTA_EN_PERIODO") # Para verificar cuántos meses tiene cada tienda
    )
)

# Ejecutar el cálculo y obtener un DataFrame de Polars
ventas_agregadas_lf = lf_venta_procesada.collect() # Ahora es un DataFrame
print("\nVentas promedio mensual calculadas para el periodo:")
print(ventas_agregadas_lf.head())
print("\nEstadísticas de N_MESES_CON_VENTA_EN_PERIODO:")
print(ventas_agregadas_lf["N_MESES_CON_VENTA_EN_PERIODO"].describe())
# Es importante ver si muchas tiendas tienen menos de 12 meses en este periodo.

In [ ]:
# --- 3. Unir (join) los DataFrames/LazyFrames ---
# Necesitamos "collect()" en los LazyFrames antes de unirlos si no lo hemos hecho
dim_tienda_train_df_collected = dim_tienda_train_lf.collect()
meta_venta_df_collected = meta_venta_lf.collect()


tiendas_con_ventas_lf = dim_tienda_train_df_collected.join(
    ventas_agregadas_lf, on="TIENDA_ID", how="left"
)

tiendas_con_metas_lf = tiendas_con_ventas_lf.join(
    meta_venta_df_collected, on="ENTORNO_DES", how="left"
)

# Manejar NaNs
# Tiendas sin ventas en el periodo seleccionado tendrán VENTA_PROMEDIO_MENSUAL = NaN
# Tiendas cuyo ENTORNO_DES no esté en Meta_venta.csv tendrán Meta_venta = NaN
print(f"\nNaNs en VENTA_PROMEDIO_MENSUAL después de joins: {tiendas_con_metas_lf['VENTA_PROMEDIO_MENSUAL'].is_null().sum()}")
print(f"NaNs en Meta_venta después de joins: {tiendas_con_metas_lf['Meta_venta'].is_null().sum()}")

tiendas_completas_lf = tiendas_con_metas_lf.drop_nulls(subset=['VENTA_PROMEDIO_MENSUAL', 'Meta_venta'])
print(f"Tiendas después de eliminar NaNs en ventas/metas: {tiendas_completas_lf.height}")


In [ ]:
# --- 4. Crear la variable EXITO ---
if tiendas_completas_lf.height > 0:
    tiendas_completas_lf = tiendas_completas_lf.with_columns(
        EXITO = (pl.col('VENTA_PROMEDIO_MENSUAL') >= pl.col('Meta_venta')).cast(pl.Int8)
    )
    print("\nVariable EXITO creada.")
    print(tiendas_completas_lf.select(['TIENDA_ID', 'VENTA_PROMEDIO_MENSUAL', 'Meta_venta', 'N_MESES_CON_VENTA_EN_PERIODO', 'EXITO']).head())
    print("\nDistribución de la variable EXITO:")
    print(tiendas_completas_lf['EXITO'].value_counts(sort=True))
else:
    print("DataFrame 'tiendas_completas_lf' está vacío. No se pudo crear la variable EXITO.")
    raise ValueError("No se pudieron generar datos completos para crear la variable EXITO.")

In [ ]:
# --- 5. Seleccionar columnas relevantes y guardar ---
columnas_a_guardar = ['TIENDA_ID', 'PLAZA_CVE', 'NIVELSOCIOECONOMICO_DES', 
                      'ENTORNO_DES', 'MTS2VENTAS_NUM', 'PUERTASREFRIG_NUM', 
                      'CAJONESESTACIONAMIENTO_NUM', 'LATITUD_NUM', 'LONGITUD_NUM', 
                      'SEGMENTO_MAESTRO_DESC', 'LID_UBICACION_TIENDA', 'EXITO',
                      'VENTA_PROMEDIO_MENSUAL', 'Meta_venta', 'N_MESES_CON_VENTA_EN_PERIODO']

columnas_existentes_final = [col for col in columnas_a_guardar if col in tiendas_completas_lf.columns]
datos_train_objetivo_final_lf = tiendas_completas_lf.select(columnas_existentes_final)

try:
    datos_train_objetivo_final_lf.write_csv(RUTA_DATOS_OUTPUT + 'datos_train_con_objetivo.csv')
    print(f"\nArchivo '{RUTA_DATOS_OUTPUT}datos_train_con_objetivo.csv' guardado exitosamente.")
    print(f"Número de tiendas en el archivo final: {datos_train_objetivo_final_lf.height}")
except Exception as e:
    print(f"Error al guardar el archivo: {e}")